## **Do NOT touch this cell**

In [21]:
# !wget http://crowley-coutaz.fr/HeadPoseDataSet/HeadPoseImageDatabase.tar.gz
# !tar -xvzf HeadPoseImageDatabase.tar.gz

import os
import glob
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data.dataset import Dataset
import torch.utils as utils
import random
import numpy as np
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'

person_list = sorted(glob.glob('Person*')) # search the folders starting with 'Person'

img_list = []
train_img_list = []
val_img_list = []
test_img_list = []
H = 288
W = 384



In [22]:
import wandb
learning_rate = 1e-5
epochs=200
wandb.init(project='Face Detection')
wandb.run.name = 'VGG16, SmoothL1, AdamW, lr='+str(learning_rate)+', epchs='+str(epochs)
wandb.run.save()
args = {
    "learning_rate": learning_rate,
    "epochs": epochs
}
wandb.config.update(args)

Training loss,█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation loss,▆▆▇█▆▆▆▇▂▁▁▂▃▂▂▂
Training loss,0.00781
Validation loss,0.00241


## **You can change the list of training and validation persons**

In [23]:
# Train & Validation Select
train_person = [0,1,2,3,4,5,6,7,8]
val_person = [9]

##**Do NOT touch this cell**

In [24]:
test_person = [10,11,12,13,14]

for i in range(len(person_list)):
    if i in train_person:
        train_img_list.extend(glob.glob(os.path.join(person_list[i], '*.jpg')))
    if i in test_person:
        test_img_list.extend(glob.glob(os.path.join(person_list[i], '*.jpg')))
    if i in val_person:
        val_img_list.extend(glob.glob(os.path.join(person_list[i], '*.jpg')))

## **You can change Transform.Compose (Data Augmentation)**

In [25]:
# Transformation with data augmentation
image_train_transform = transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5612, 0.5684, 0.5908),(0.2566,0.2496, 0.2446)),
])



## **Do NOT touch this cell**

In [26]:
image_test_transform = transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5612, 0.5684, 0.5908),(0.2566,0.2496, 0.2446)),
])

image_inverse_transform = transforms.Compose([
                                              transforms.Normalize((-2.1868,-2.2768,-2.4146),(3.8966,4.0052,4.0869)),
])

## **Do NOT touch this cell**

In [27]:
############################################################################
#
#                   Data set and dataloader
#
############################################################################

class custom_dataset():
    def __init__(self, img_list, is_train):
        self.img_list = img_list
        self.is_train = is_train

    def __getitem__(self,index):
        img_file = self.img_list[index]

        image = cv2.imread(img_file) # read the image file
        image = Image.fromarray(image)
        txt_file = img_file[:-4] + '.txt' # take the txt file address from the image file address
        with open(txt_file,'r') as f:
            line = f.read().splitlines()
            center_x = int(line[3])/W
            center_y = int(line[4])/H
            width = int(line[5])/W
            height = int(line[6])/H # open the text file and take the face box information from it
        if self.is_train:
            image = image_train_transform(image)
        else:
            image = image_test_transform(image)
        label = torch.tensor([center_x, center_y, width, height])

        return image, label

    def __len__(self):
        return len(self.img_list)

train_datasets = custom_dataset(train_img_list, is_train=True)
val_datasets = custom_dataset(val_img_list, is_train=False)
test_datasets = custom_dataset(test_img_list, is_train=False)

trainloader = torch.utils.data.DataLoader(train_datasets, batch_size=8, shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(val_datasets, batch_size=8, shuffle=False, num_workers=2)
testloader = torch.utils.data.DataLoader(test_datasets, batch_size=1, shuffle=False, num_workers=2)


############################################################################
#
#                   Sample visualization
#
############################################################################

sample_image, sample_label = next(iter(trainloader))

sample_image = sample_image[0]
sample_label = sample_label[0]

sample_image = image_inverse_transform(sample_image)

sample_cx = sample_label[0] * W
sample_cy = sample_label[1]* H
sample_w = sample_label[2]* W
sample_h = sample_label[3] * H

# calculate the left-top & right-down coordinates from the box info( x, y, w, h)
sample_x1 = sample_cx - torch.div(sample_w, 2, rounding_mode='floor')
sample_y1 = sample_cy - torch.div(sample_h, 2, rounding_mode='floor')
sample_x2 = sample_cx + torch.div(sample_w, 2, rounding_mode='floor')
sample_y2 = sample_cy + torch.div(sample_h, 2, rounding_mode='floor')

sample_image = sample_image.numpy().transpose(1,2,0).copy() * 255
print('               ( X,   Y)')
print('Top Left     : (%3d,%3d)'%(sample_x1,sample_y1))
print('Bottom Right : (%3d,%3d)'%(sample_x2,sample_y2))

sample_image = cv2.rectangle(sample_image, (int(sample_x1), int(sample_y1)), (int(sample_x2),int(sample_y2)), (255,0,0), (1)) # draw blue rectangle on the image
# sample_image = cv2.UMat.get(sample_image)


               ( X,   Y)
Top Left     : ( 14,110)
Bottom Right : (118,260)


## **You have to make your own network.**

In [ ]:
# class FaceNet(nn.Module):
#     def __init__(self, H=288, W=384, num_filters=16, padding='same', pool='max', dropout_rate=0.2):
#         super(FaceNet, self).__init__()
#         ###############################
#         if padding == 'valid':
#             padding_size = 0
#         elif padding == 'same':
#             padding_size = 1

#         if pool == 'max':
#             pool_layer = nn.MaxPool2d(kernel_size=2, stride=2)
#         elif pool == 'avg':
#             pool_layer = nn.AvgPool2d(kernel_size=2, stride=2)

#         self.layer1 = torch.nn.Sequential(
#             torch.nn.Conv2d(3, num_filters, kernel_size=3, stride=1, padding=padding_size),
#             nn.BatchNorm2d(num_filters),
#             torch.nn.ReLU(),
#             pool_layer)                                                                             # (num_filters, H/2, W/2)
#         self.layer2 = torch.nn.Sequential(
#             torch.nn.Conv2d(num_filters, num_filters*2, kernel_size=3, stride=1, padding=padding_size),
#             nn.BatchNorm2d(num_filters*2),
#             torch.nn.ReLU(),
#             pool_layer)                                                                             # (num_filters*2, H/4, W/4)
#         self.layer3 = torch.nn.Sequential(
#             torch.nn.Conv2d(num_filters*2, num_filters*4, kernel_size=3, stride=1, padding=padding_size),
#             nn.BatchNorm2d(num_filters*4),
#             torch.nn.ReLU(),
#             pool_layer)                                                                             # (num_filters*4, H/8, W/8)
#         self.layer3 = torch.nn.Sequential(
#             torch.nn.Conv2d(num_filters*2, num_filters*4, kernel_size=3, stride=1, padding=padding_size),
#             nn.BatchNorm2d(num_filters*4),
#             torch.nn.ReLU(),
#             pool_layer)
#         self.feature = nn.Sequential(
#             self.layer1,
#             self.layer2,
#             self.layer3
#         )

#         self.fc1 = torch.nn.Linear(num_filters*4*(H//8)*(W//8), num_filters*2, bias=True)
#         torch.nn.init.xavier_uniform_(self.fc1.weight)
#         self.layer4 = torch.nn.Sequential(
#             self.fc1,
#             torch.nn.ReLU(),
#             torch.nn.Dropout(dropout_rate))
#         self.fc2 = torch.nn.Linear(num_filters*2, 4, bias=True)
#         torch.nn.init.xavier_uniform_(self.fc2.weight)
#         self.classifier = nn.Sequential(
#             self.layer4,
#             self.fc2
#         )
#         ###############################

#     def forward(self, x):
#         x = self.feature(x)
#         x = x.view(x.size(0), -1)
#         x = self.classifier(x)
#         return x

cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
    'DEBUG' : [64, 'M']
}

class FaceNet(nn.Module):
    def __init__(self, vgg_name='VGG16', H=288, W=384):
        super(FaceNet, self).__init__()
        ###############################
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Sequential(
            nn.Linear(512 * H//32 * W//32, 360),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(360, 100),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(100, 4),
        )

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out
    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        return nn.Sequential(*layers)

print('Building Network!')
net = FaceNet()

net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.SmoothL1Loss()
# criterion = nn.MSELoss()
# optimizer = optim.SGD(net.parameters(), momentum=0.9, weight_decay=5e-4, lr=1e-4)
optimizer = optim.AdamW(net.parameters(), lr=learning_rate)
# scheduler = optim.lr_scheduler.LambdaLR(optimizer=optimizer,
#                                         lr_lambda=lambda epoch: 0.95 ** epoch)
# epochs = 200

def train(epoch):
    print('\nTrain:')
    net.train()
    train_loss = 0
    for batch_idx, (images, targets) in enumerate(trainloader):
        optimizer.zero_grad()
        images, targets = images.to(device), targets.to(device)
        outputs = net(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        if ((batch_idx+1) % (len(trainloader)/3) == 1) or batch_idx+ 1 == len(trainloader):
            print('[%3d/%3d] | Loss: %.5f'%(batch_idx+1, len(trainloader), train_loss/(batch_idx+1)))
            wandb.log({"Training loss": train_loss/(batch_idx+1)})
    # print('[%3d/%3d] | Loss: %.5f'%(batch_idx+1, len(trainloader), train_loss/(batch_idx+1)))

def val(epoch):
    print('\nValidation:')
    net.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(valloader):

            optimizer.zero_grad()
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)

            loss = criterion(outputs, labels)
            val_loss += loss.item()
            # scheduler.step()

            if ((batch_idx+1) % (len(valloader)/3) == 1) or batch_idx+ 1 == len(valloader):
                print('[%3d/%3d] | Loss: %.5f'%(batch_idx+1, len(valloader), val_loss/(batch_idx+1)))
                wandb.log({"Validation loss": val_loss/(batch_idx+1)})
        # print('[%3d/%3d] | Loss: %.5f'%(batch_idx+1, len(valloader), val_loss/(batch_idx+1)))

for epoch in range(epochs):
    print('\nEpoch %d'%(epoch))
    train(epoch)
    val(epoch)

Building Network!

Epoch 0

Train:
[  1/210] | Loss: 0.09994
[ 71/210] | Loss: 0.02351
[141/210] | Loss: 0.01975
[210/210] | Loss: 0.01794

Validation:
[  1/ 24] | Loss: 0.00272
[  9/ 24] | Loss: 0.00386
[ 17/ 24] | Loss: 0.00434
[ 24/ 24] | Loss: 0.00446

Epoch 1

Train:
[  1/210] | Loss: 0.01456
[ 71/210] | Loss: 0.01373
[141/210] | Loss: 0.01357
[210/210] | Loss: 0.01298

Validation:
[  1/ 24] | Loss: 0.00192
[  9/ 24] | Loss: 0.00245
[ 17/ 24] | Loss: 0.00261
[ 24/ 24] | Loss: 0.00272

Epoch 2

Train:
[  1/210] | Loss: 0.01718
[ 71/210] | Loss: 0.01227
[141/210] | Loss: 0.01247
[210/210] | Loss: 0.01221

Validation:
[  1/ 24] | Loss: 0.00199
[  9/ 24] | Loss: 0.00312
[ 17/ 24] | Loss: 0.00322
[ 24/ 24] | Loss: 0.00333

Epoch 3

Train:
[  1/210] | Loss: 0.01807
[ 71/210] | Loss: 0.01183
[141/210] | Loss: 0.01120
[210/210] | Loss: 0.01078

Validation:
[  1/ 24] | Loss: 0.00359
[  9/ 24] | Loss: 0.00341
[ 17/ 24] | Loss: 0.00354
[ 24/ 24] | Loss: 0.00361

Epoch 4

Train:
[  1/210] | L

##**Do NOT touch this cell**

In [ ]:
fig = plt.figure(6, 300)

visualize_idx = np.random.randint(len(testloader), size=100)
cnt = 0
test_loss = 0


with torch.no_grad():
    print('\nTest:')
    net.eval()
    for batch_idx, (images, labels) in enumerate(testloader):
        optimizer.zero_grad()
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        if batch_idx+1 == len(testloader):
            print('[%3d/%3d] | final_MSE: %f'%(batch_idx+1, len(testloader), test_loss/(batch_idx+1)))
            print('\033[31m' + '[GT □]' + '\033[30m' +' | ' + '\033[34m' +'[Pred □]' +  '\033[0m')
        if batch_idx in visualize_idx: # visualize for selected images
            subplot = fig.add_subplot(2,50, cnt+1)

            sample_image = images[0]
            sample_outputs = outputs[0]
            sample_label = labels[0]

            sample_image = image_inverse_transform(sample_image)
            sample_image = sample_image.cpu().numpy().transpose(1,2,0) * 255



            # Draw for predicted box
            sample_cx = sample_outputs[0] * W
            sample_cy = sample_outputs[1] * H
            sample_w = sample_outputs[2] * W
            sample_h = sample_outputs[3] * H

            sample_x1 = sample_cx - torch.div(sample_w, 2, rounding_mode='floor')
            sample_y1 = sample_cy - torch.div(sample_h, 2, rounding_mode='floor')
            sample_x2 = sample_cx + torch.div(sample_w, 2, rounding_mode='floor')
            sample_y2 = sample_cy + torch.div(sample_h, 2, rounding_mode='floor')

            sample_image = cv2.rectangle(sample_image.copy(), (int(sample_x1), int(sample_y1)), (int(sample_x2),int(sample_y2)), (255, 0, 0), (3))
            # sample_image = cv2.UMat.get(sample_image)

            # Draw for groundtruth box
            sample_cx = sample_label[0] * W
            sample_cy = sample_label[1] * H
            sample_w = sample_label[2] * W
            sample_h = sample_label[3] * H

            sample_x1 = sample_cx - torch.div(sample_w, 2, rounding_mode='floor')
            sample_y1 = sample_cy - torch.div(sample_h, 2, rounding_mode='floor')
            sample_x2 = sample_cx + torch.div(sample_w, 2, rounding_mode='floor')
            sample_y2 = sample_cy + torch.div(sample_h, 2, rounding_mode='floor')

            sample_image = cv2.rectangle(sample_image.copy(), (int(sample_x1), int(sample_y1)), (int(sample_x2),int(sample_y2)), (0, 0, 255), (3))

            subplot.set_xticks([])
            subplot.set_yticks([])
            subplot.set_title('%f'%loss.item())
            subplot.imshow(cv2.cvtColor(sample_image.astype('uint8'), cv2.COLOR_BGR2RGB))
            cnt += 1

plt.show()